In [9]:
import pyodbc

conn = pyodbc.connect('Driver={SQL Server};''Server=172.29.81.133;''Database=ComparisonData;''Trusted_Connection=yes;')
import pandas as pd

In [10]:
companyrelID=70389560

In [11]:
qd1="""select  KeyInstn from RelMap..companyrelfinal_tbl  a  where companyrelID ="""+str(companyrelID)+""" --and    a.updoperation  in ('I','U')   and a.winnerflag =1 and isnull(a.appstatus ,2) <>-1"""
qd2="""select  KeyInstnparent        from RelMap..companyrelfinal_tbl  a  where companyrelID ="""+str(companyrelID)+""""""
KI=pd.read_sql(qd1, conn)['KeyInstn'][0]
KIParent=pd.read_sql(qd2, conn)['KeyInstnparent'][0]

In [12]:
querydoc1="""Select 'CRF' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived',RelCategoryID, WinnerFlag as 'WF', Upddate,KeyInstn, KeyInstnParent, AppStatus,  CompanyRelFinalID, CompanyRelTypeID, CompanyID2, CompanyID, CompanyRelID, StartDate, Enddate, StakeTypeId, Percentownership, KeyInstnMultipleParents as 'KIMP', UpdOperation as upd, ControllingParentFlag as 'CP' , InstnOwnerFlag as 'IO',old_keyinstn, old_keyinstnparent,old_companyid2,old_companyid FROM Relmap..CompanyRelFinal_TBL (NOLOCK) C WHERE companyrelID = """+str(companyrelID)+""" ORDER BY UPDDATE ASC"""
querydoc3="""Select TOP 25 'Comprel_IMPH' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived', KeyInstn, KeyInstnParent, PercentOwned, InstnParentAsOf ,AppStatus ,UpdOperation ,UpdDate ,KeyInstnMultipleParents as 'KIMP' ,InstnParentEndedAsOf, InstnParentGeneralPartner as 'CP', InstnParentInstitutional as 'IO' , CompanyRelTypeId , stakeTypeId , KeyIMPHDateSourceAsOf, KeyIMPHDateSourceAsOfEnded FROM compRelMap..Instnmultipleparenthistory (NOLOCK) WHERE KeyInstn = """+str(KI)+""" AND KeyInstnParent IS NOT NULL AND KeyInstnParent = """+str(KIParent)+""" ORDER BY UPDDATE ASC --AND KeyInstnParent = --KeyInstnmultipleparents = 110455006"""
querydoc4="""select  'ImpLatest' as 'Table', KeyMINameSpace,          KeyIMPHRelationshipDerived, InstnParentAsOfAnalystEntered,KeyInstn     ,KeyInstnParent,* from relmap..implatest where keyinstn= """+str(KI)+"""   and keyinstnparent =  """+str(KIParent)+""""""
querydoc2="""Select TOP 25 'SNLIMPH' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived', KeyInstn, KeyInstnParent, PercentOwned, InstnParentAsOf ,AppStatus ,UpdOperation ,UpdDate ,KeyInstnMultipleParents as 'KIMP' ,InstnParentEndedAsOf, InstnParentGeneralPartner as 'CP', InstnParentInstitutional as 'IO' , CompanyRelTypeId , stakeTypeId , KeyIMPHDateSourceAsOf, KeyIMPHDateSourceAsOfEnded FROM compRelMap..SNLInstnmultipleparenthistory  (NOLOCK) WHERE KeyInstn = """+str(KI)+""" AND KeyInstnParent IS NOT NULL AND KeyInstnParent = """+str(KIParent)+""" ORDER BY UPDDATE ASC --AND KeyInstnParent = --KeyInstnmultipleparents = 110455006"""

df1=pd.read_sql(querydoc1, conn)
df2=pd.read_sql(querydoc2, conn)
df3=pd.read_sql(querydoc3, conn)
df4=pd.read_sql(querydoc4, conn)

In [13]:
df2

,Table,AnalystEntered,KeyMINamespace,RelDerived,KeyInstn,KeyInstnParent,PercentOwned,InstnParentAsOf,AppStatus,UpdOperation,UpdDate,KIMP,InstnParentEndedAsOf,CP,IO,CompanyRelTypeId,stakeTypeId,KeyIMPHDateSourceAsOf,KeyIMPHDateSourceAsOfEnded
0,SNLIMPH,2015-05-06,1,NaN,4604537,4044305,100.0,2015-05-06,2,2,2021-03-22 03:36:31.960,2105071435,None,1,False,0,0,NaN,None
1,SNLIMPH,2015-05-06,1,2.0,4604537,4044305,NaN,2017-10-11,2,1,2021-03-22 07:56:40.637,2105071555,2020-01-10,1,False,9,0,NaN,None
2,SNLIMPH,2015-05-06,1,2.0,4604537,4044305,100.0,2016-05-06,2,1,2021-03-22 07:56:40.637,555606,2017-10-10,1,False,0,0,1.0,None


In [14]:
import pandas as pd
import os

def multiple_dfs(df_list, sheets, file_name, spaces):
    arg_mode = 'w' #'a' if file_name in os.getcwd() else 'w' # line added
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter', mode=arg_mode) # added mode argument
    row = 0

    for dataframe in df_list:
        dataframe.to_excel(writer,sheet_name=sheets,startrow=row , startcol=0)   
        row = row + len(dataframe.index) + spaces + 1
    writer.save()

In [15]:
df_list=[df1,df2,df3,df3]
multiple_dfs(df_list,'Sheet1', 'test1.xlsx', 2)

FileCreateError: [Errno 13] Permission denied: 'test1.xlsx'

In [ ]:
Declare @companyrelID int,@KI int ,@KIParent  int
set @companyrelID=70389560
set @KI  = (select  KeyInstn              from RelMap..companyrelfinal_tbl  a  where companyrelID =@companyrelID) --and    a.updoperation  in ('I','U')   and a.winnerflag =1and isnull(a.appstatus ,2) <>-1)
Set @KIParent= (select  KeyInstnparent        from RelMap..companyrelfinal_tbl  a  where companyrelID =@companyrelID)   --and  a.updoperation  in ('I','U')   and a.winnerflag =1and isnull(a.appstatus ,2) <>-1)

Select 'CRF' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived',RelCategoryID, WinnerFlag as WF, Upddate,KeyInstn, KeyInstnParent, AppStatus,  CompanyRelFinalID, CompanyRelTypeID, CompanyID2, CompanyID, CompanyRelID, StartDate, Enddate, StakeTypeId, Percentownership, KeyInstnMultipleParents as 'KIMP', UpdOperation as upd, ControllingParentFlag as 'CP' , InstnOwnerFlag as 'IO',old_keyinstn, old_keyinstnparent,old_companyid2,old_companyid FROM Relmap..CompanyRelFinal_TBL (NOLOCK) C WHERE companyrelID = @companyrelID ORDER BY UPDDATE ASC

Select  'Stagebkup' as 'Table Name', InstnParentAsOfAnalystEntered, * FROM relmap..stgbkp_crfpart123 WHERE KeyInstn = 9720167 AND KeyInstnParent = 9704406 ORDER BY UpdDate DESC

Select 'Stage' as 'Table Name', InstnParentAsOfAnalystEntered ,     KeyMINamespace    ,KeyIMPHRelationshipDerived as 'RelDerived'  ,KeyInstn    ,KeyInstnParent    ,InstnParentAsOf    ,InstnParentEndedAsOf    ,KeyInstnMultipleParents    ,InstnParentGeneralPartner,    InstnParentInstitutional,    UpdDate    ,AppStatus    ,UpdOperation    ,PercentOwned    ,KeyCompanyRelationshipType    ,KeyOwnershipStakeType,'', StatusFlag from Relmap..StageIntegratedCompanyRel_tbl   (NOLOCK)   WHERE  KeyInstn = @KI

Select TOP 25 'SNLIMPH' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived', KeyInstn, KeyInstnParent, PercentOwned, InstnParentAsOf ,AppStatus ,UpdOperation ,UpdDate ,KeyInstnMultipleParents as 'KIMP' ,InstnParentEndedAsOf, InstnParentGeneralPartner as 'CP', InstnParentInstitutional as 'IO' , CompanyRelTypeId , stakeTypeId , KeyIMPHDateSourceAsOf, KeyIMPHDateSourceAsOfEnded FROM compRelMap..SNLInstnmultipleparenthistory  (NOLOCK) WHERE KeyInstn = @KI AND KeyInstnParent IS NOT NULL AND KeyInstnParent = @KIParent ORDER BY UPDDATE ASC--AND KeyInstnParent = --KeyInstnmultipleparents = 110455006

Select TOP 25 'Comprel_IMPH' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived', KeyInstn, KeyInstnParent, PercentOwned, InstnParentAsOf ,AppStatus ,UpdOperation ,UpdDate ,KeyInstnMultipleParents as 'KIMP' ,InstnParentEndedAsOf, InstnParentGeneralPartner as 'CP', InstnParentInstitutional as 'IO' , CompanyRelTypeId , stakeTypeId , KeyIMPHDateSourceAsOf, KeyIMPHDateSourceAsOfEnded FROM compRelMap..Instnmultipleparenthistory (NOLOCK) WHERE KeyInstn = @KI AND KeyInstnParent IS NOT NULL AND KeyInstnParent = @KIParent ORDER BY UPDDATE ASC--AND KeyInstnParent = --KeyInstnmultipleparents = 110455006

select  'ImpLatest', KeyMINameSpace,          KeyIMPHRelationshipDerived, InstnParentAsOfAnalystEntered,KeyInstn     ,KeyInstnParent,* from relmap..implatest where keyinstn= @KI   and keyinstnparent =  @KIParent



In [ ]:
Declare @companyrelID int,@KI int ,@KIParent  int
set @companyrelID=70389560
set @KI  = (select  KeyInstn              from RelMap..companyrelfinal_tbl  a  where companyrelID =@companyrelID) --and    a.updoperation  in ('I','U')   and a.winnerflag =1and isnull(a.appstatus ,2) <>-1)
Set @KIParent= (select  KeyInstnparent        from RelMap..companyrelfinal_tbl  a  where companyrelID =@companyrelID)   --and  a.updoperation  in ('I','U')   and a.winnerflag =1and isnull(a.appstatus ,2) <>-1)

----Select Top 100 'Comparisondata..CompanyRel_TBL' as 'Table ', * FROM Comparisondata..CompanyRel_TBL (NOLOCK) C WHERE companyrelID = @companyrelID
--Select Top 100 'Comparisondata..CompanyRelInfo_TBL' as 'Table ', CompanyRelInfoID, CompanyRelID, PercentOwnership, StakeTypeId, PPOUpdateTypeId, usercompanyid FROM Comparisondata..CompanyRelInfo_TBL (NOLOCK) CI WHERE companyrelID = @companyrelID
----Select Top 100 'CMPREL_TBL' as 'Table ', * FROM Comparisondata_relmap..CompanyRel_TBL (NOLOCK) C WHERE companyrelID = @companyrelID
--Select Top 100 'RELINFO' as 'Table ', CompanyRelInfoID, CompanyRelID, PercentOwnership, StakeTypeId, PPOUpdateTypeId, usercompanyid FROM Comparisondata_relmap..CompanyRelInfo_TBL (NOLOCK) CI WHERE companyrelID = @companyrelID
--Select 'REL_TMP' as 'Table ',StatusFlag, * FROM Relmap..CompanyRelTMP_TBL (NOLOCK) CRT WHERE companyrelID = @companyrelID ORDER BY UPDDATE DESC
Select 'RELINFO_TMP' as 'Table ', StatusFlag, * FROM Relmap..CompanyRelInfoTMP_TBL (NOLOCK) CRT WHERE companyrelID = @companyrelID ORDER BY UPDDATE DESC

Select 'CRF' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived',RelCategoryID, WinnerFlag as WF, Upddate,KeyInstn, KeyInstnParent, AppStatus,  CompanyRelFinalID, CompanyRelTypeID, CompanyID2, CompanyID, CompanyRelID, StartDate, Enddate, StakeTypeId, Percentownership, KeyInstnMultipleParents as 'KIMP', UpdOperation as upd, ControllingParentFlag as 'CP' , InstnOwnerFlag as 'IO',old_keyinstn, old_keyinstnparent,old_companyid2,old_companyid FROM Relmap..CompanyRelFinal_TBL (NOLOCK) C WHERE companyrelID = @companyrelID ORDER BY UPDDATE ASC
--Select InstnParentAsOfAnalystEntered, * FROM relmap..stgbkp_crfpart123 WHERE KeyInstn = 9720167 AND KeyInstnParent = 9704406 ORDER BY UpdDate DESC
Select 'Stage' as 'Table Name', InstnParentAsOfAnalystEntered ,     KeyMINamespace    ,KeyIMPHRelationshipDerived as 'RelDerived'  ,KeyInstn    ,KeyInstnParent    ,InstnParentAsOf    ,InstnParentEndedAsOf    ,KeyInstnMultipleParents    ,InstnParentGeneralPartner,    InstnParentInstitutional,    UpdDate    ,AppStatus    ,UpdOperation    ,PercentOwned    ,KeyCompanyRelationshipType    ,KeyOwnershipStakeType,'', StatusFlag from Relmap..StageIntegratedCompanyRel_tbl   (NOLOCK)   WHERE  KeyInstn = @KI
--Select 'fi' AS 'tbl',  InstnParentAsOfAnalystEntered ,     KeyMINamespace    ,KeyIMPHRelationshipDerived as 'RelDerived'  ,KeyInstn    ,KeyInstnParent    ,InstnParentAsOf    ,InstnParentEndedAsOf    ,KeyInstnMultipleParents    ,InstnParentGeneralPartner,    InstnParentInstitutional,    UpdDate    ,AppStatus    ,UpdOperation    ,PercentOwned    ,KeyCompanyRelationshipType    ,KeyOwnershipStakeType FROM Relmap..FinalIntegratedCompanyRel_TBL (NOLOCK) WHERE KeyInstn = @KI AND KeyInstnParent IS NOT NULL ORDER BY UPDDATE DESC
Select TOP 25 'SNLIMPH' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived', KeyInstn, KeyInstnParent, PercentOwned, InstnParentAsOf ,AppStatus ,UpdOperation ,UpdDate ,KeyInstnMultipleParents as 'KIMP' ,InstnParentEndedAsOf, InstnParentGeneralPartner as 'CP', InstnParentInstitutional as 'IO' , CompanyRelTypeId , stakeTypeId , KeyIMPHDateSourceAsOf, KeyIMPHDateSourceAsOfEnded FROM compRelMap..SNLInstnmultipleparenthistory  (NOLOCK) WHERE KeyInstn = @KI AND KeyInstnParent IS NOT NULL AND KeyInstnParent = @KIParent ORDER BY UPDDATE ASC--AND KeyInstnParent = --KeyInstnmultipleparents = 110455006
Select TOP 25 'Comprel_IMPH' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived', KeyInstn, KeyInstnParent, PercentOwned, InstnParentAsOf ,AppStatus ,UpdOperation ,UpdDate ,KeyInstnMultipleParents as 'KIMP' ,InstnParentEndedAsOf, InstnParentGeneralPartner as 'CP', InstnParentInstitutional as 'IO' , CompanyRelTypeId , stakeTypeId , KeyIMPHDateSourceAsOf, KeyIMPHDateSourceAsOfEnded FROM compRelMap..Instnmultipleparenthistory (NOLOCK) WHERE KeyInstn = @KI AND KeyInstnParent IS NOT NULL AND KeyInstnParent = @KIParent ORDER BY UPDDATE ASC--AND KeyInstnParent = --KeyInstnmultipleparents = 110455006
select 'Fakearm' errordesc, keyinstn, keyinstnparent,companyrelid,* from  Relmap..ExcpnFakearms_TBL    (NOLOCK) where companyrelid =  @companyrelID

Select 'STG_bkp' as 'Table Name', InstnParentAsOfAnalystEntered ,     KeyMINamespace    ,KeyIMPHRelationshipDerived    ,KeyInstn    ,KeyInstnParent    ,InstnParentAsOf   , PercentOwned,InstnParentEndedAsOf    ,KeyInstnMultipleParents    ,InstnParentGeneralPartner,    InstnParentInstitutional,    UpdDate    ,AppStatus    ,UpdOperation    ,PercentOwned    ,KeyCompanyRelationshipType    ,KeyOwnershipStakeType,'', StatusFlag from relmap..stgbkp_crfpart123   (NOLOCK)   WHERE  KeyInstn = @KI AND KeyInstnParent = @KIParent ORDER BY UPDDATE ASC
--select  'ImpLatest_Crf', KeyMINameSpace,          KeyIMPHRelationshipDerived, InstnParentAsOfAnalystEntered,KeyInstn     ,KeyInstnParent,* from relmap..implatest_crf where keyinstn= @KI   and keyinstnparent =  @KIParent
select  'ImpLatest', KeyMINameSpace,          KeyIMPHRelationshipDerived, InstnParentAsOfAnalystEntered,KeyInstn     ,KeyInstnParent,* from relmap..implatest where keyinstn= @KI   and keyinstnparent =  @KIParent
sELECT 'Excpn',errordescid,Keyinstn, Keyinstnparent,* FROM relmap..excpncompanyrel_tbl (nolock) WHERE companyrelID = @companyrelID 

In [ ]:
Declare @KI int ,@KIParent  int
set @KI  = 4724394	
Set @KIParent= 4229491 	

Select 'CRF' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived',Upddate,RelCategoryID, WinnerFlag as WF, KeyInstn, KeyInstnParent, AppStatus,  CompanyRelFinalID, CompanyRelTypeID, CompanyID2, CompanyID, CompanyRelID, StartDate, Enddate, StakeTypeId, Percentownership, KeyInstnMultipleParents as 'KIMP', UpdOperation as upd, ControllingParentFlag as 'CP' , InstnOwnerFlag as 'IO',old_keyinstn, old_keyinstnparent 
FROM Relmap..CompanyRelFinal_TBL (NOLOCK) C WHERE KeyInstn = @KI AND KeyInstnParent = @KIParent  ORDER BY UPDDATE DESC

Select TOP 25 'SNLIMPH' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived', KeyInstn, KeyInstnParent, PercentOwned, InstnParentAsOf ,AppStatus ,UpdOperation ,UpdDate ,KeyInstnMultipleParents as 'KIMP' ,InstnParentEndedAsOf, InstnParentGeneralPartner as 'CP', InstnParentInstitutional as 'IO' , CompanyRelTypeId , stakeTypeId , KeyIMPHDateSourceAsOf, KeyIMPHDateSourceAsOfEnded FROM compRelMap..SNLInstnmultipleparenthistory  (NOLOCK) 
WHERE KeyInstn = @KI AND KeyInstnParent = @KIParent ORDER BY UPDDATE DESC--AND KeyInstnParent = --KeyInstnmultipleparents = 110455006

Select TOP 25 'COMPIMPH' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived', KeyInstn, KeyInstnParent, PercentOwned, InstnParentAsOf ,AppStatus ,UpdOperation ,UpdDate ,KeyInstnMultipleParents as 'KIMP' ,InstnParentEndedAsOf, InstnParentGeneralPartner as 'CP', InstnParentInstitutional as 'IO' , CompanyRelTypeId , stakeTypeId , KeyIMPHDateSourceAsOf, KeyIMPHDateSourceAsOfEnded FROM compRelMap..Instnmultipleparenthistory  (NOLOCK) 
WHERE KeyInstn = @KI AND KeyInstnParent = @KIParent ORDER BY UPDDATE DESC--AND KeyInstnParent = --KeyInstnmultipleparents = 110455006

Select 'Stage' as 'Table Name', InstnParentAsOfAnalystEntered ,     KeyMINamespace    ,KeyIMPHRelationshipDerived as 'RelDerived'    ,KeyInstn    ,KeyInstnParent   PercentOwned ,InstnParentAsOf    ,InstnParentEndedAsOf    ,KeyInstnMultipleParents    ,InstnParentGeneralPartner,    InstnParentInstitutional,    UpdDate    ,AppStatus    ,UpdOperation    ,PercentOwned    ,KeyCompanyRelationshipType    ,KeyOwnershipStakeType,'','','','','','','','', StatusFlag from Relmap..StageIntegratedCompanyRel_tbl   (NOLOCK)   WHERE  KeyInstn = @KI
Select 'STG_bkp' as 'Table Name', InstnParentAsOfAnalystEntered ,     KeyMINamespace    ,KeyIMPHRelationshipDerived    ,KeyInstn    ,KeyInstnParent   PercentOwned ,InstnParentAsOf    ,InstnParentEndedAsOf    ,KeyInstnMultipleParents    ,InstnParentGeneralPartner,    InstnParentInstitutional,    UpdDate    ,AppStatus    ,UpdOperation    ,PercentOwned    ,KeyCompanyRelationshipType    ,KeyOwnershipStakeType,'','','','','','','','', StatusFlag from relmap..stgbkp_crfpart123   (NOLOCK)   WHERE  KeyInstn = @KI AND KeyInstnParent = @KIParent ORDER BY UPDDATE ASC

Select TOP 25 'FI' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived', KeyInstn, KeyInstnParent, PercentOwned, InstnParentAsOf ,AppStatus ,UpdOperation ,UpdDate ,KeyInstnMultipleParents as 'KIMP' ,InstnParentEndedAsOf, InstnParentGeneralPartner as 'CP', InstnParentInstitutional as 'IO' , KeyCompanyRelationshipType ,KeyOwnershipStakeType FROM relmap..Finalintegratedcompanyrel_Tbl  (NOLOCK) 
WHERE KeyInstn = @KI AND KeyInstnParent = @KIParent ORDER BY UPDDATE DESC--AND KeyInstnParent = --KeyInstnmultipleparents = 110455006
sELECT 'Excpn',* FROM relmap..excpncompanyrel_tbl (nolock) WHERE KeyInstn = @KI AND KeyInstnParent = @KIParent
select  'ImpLatest', KeyMINameSpace,          KeyIMPHRelationshipDerived, InstnParentAsOfAnalystEntered,KeyInstn     ,KeyInstnParent,* from relmap..implatest where keyinstn= @KI   and keyinstnparent =  @KIParent
select 'Fakearm' errordesc, keyinstn, keyinstnparent,companyrelid,* from  Relmap..ExcpnFakearms_TBL    (NOLOCK) WHERE KeyInstn = @KI AND KeyInstnParent = @KIParent


In [ ]:
Declare @KI int 
set @KI=4061430

select  'CRF' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived',RelCategoryID, WinnerFlag as WF, KeyInstn, KeyInstnParent, AppStatus,  CompanyRelFinalID, CompanyRelTypeID, CompanyID2, CompanyID, CompanyRelID, StartDate, Enddate, StakeTypeId, Percentownership, KeyInstnMultipleParents as 'KIMP', UpdOperation as upd, ControllingParentFlag as 'CP' , InstnOwnerFlag as 'IO',old_keyinstn, old_keyinstnparent , Old_companyid2, old_companyid
from relmap..companyrelfinal_tbl (nolock) 
Where keyinstn = @KI

Select 'EXCPN' as 'Table' ,errordescid,keyinstn, keyinstnparent,* from relmap..excpncompanyrel_tbl (nolock)
Where keyinstn =  @KI

select 'stage' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived', KeyInstn, KeyInstnParent, PercentOwned, InstnParentAsOf ,* from Relmap..StageIntegratedCompanyRel_tbl   (NOLOCK)
where keyinstn=  @KI

Select 'SNLIMPH' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived', KeyInstn, KeyInstnParent, PercentOwned, InstnParentAsOf ,AppStatus ,UpdOperation ,UpdDate ,KeyInstnMultipleParents as 'KIMP' ,InstnParentEndedAsOf, InstnParentGeneralPartner as 'CP', InstnParentInstitutional as 'IO' , CompanyRelTypeId , stakeTypeId , KeyIMPHDateSourceAsOf, KeyIMPHDateSourceAsOfEnded 
 from Comprelmap..SNLinstnmultipleparenthistory (nolock)
where keyinstn =  @KI

Select 'SNLIMPH' as 'Table', InstnParentAsOfAnalystEntered as 'AnalystEntered', KeyMINamespace, KeyIMPHRelationshipDerived as 'RelDerived', KeyInstn, KeyInstnParent, PercentOwned, InstnParentAsOf ,AppStatus ,UpdOperation ,UpdDate ,KeyInstnMultipleParents as 'KIMP' ,InstnParentEndedAsOf, InstnParentGeneralPartner as 'CP', InstnParentInstitutional as 'IO' , CompanyRelTypeId , stakeTypeId , KeyIMPHDateSourceAsOf, KeyIMPHDateSourceAsOfEnded 
from Comprelmap..instnmultipleparenthistory (nolock)
where keyinstn =  @KI

select  'ImpLatest', KeyMINameSpace,          KeyIMPHRelationshipDerived, InstnParentAsOfAnalystEntered,KeyInstn     ,KeyInstnParent,* from relmap..implatest
where keyinstn=  @KI
